In [1]:
%load_ext autoreload
%autoreload 2

import FiniteVolumeGroups as fvg
from su4ops.quark import Quark,Gamma,GammaProper
from su4ops.elemental import Elemental
from su4ops.group_rep_utils import *
from su4ops.utils import *
from su4ops.constants import *

NC=4

In [2]:
oh=fvg.cubic.Oh()

In [3]:
uQ = lambda spin : Quark({'bar': False, 'flavor': 0, 'color': 0, 'spin': spin})
uQBar = lambda spin: Quark({'bar': True, 'flavor': 0, 'color': 0, 'spin': spin})

In [4]:
allSpins=[]
basis=[]
allSpinsToBasis={}
basisIdx=0
for i in range(NS**NC):
    quarks=[]
    tmp=i
    for s in range(NC):
        quarks.append(uQBar(tmp%NS))
        tmp=tmp//NS
    # match order according to equation 4.66
    quarks[0], quarks[2] = quarks[2], quarks[0]
    # add the new elemental to allSpins
    allSpins.append(Elemental(1,quarks))
    # now check if it's already in the basis
    perms = []
    for p in permutations(quarks):
        perms.append(copy.deepcopy(p))

    found=False
    for b in basis:
        if b.quarks in perms:
            found=True
            foundIdx=basis.index(b)

    if not found:
        basis.append(Elemental(1,quarks))
        allSpinsToBasis[i]=basisIdx
        basisIdx+=1
    else:
        allSpinsToBasis[i]=foundIdx

#for b in basis:
    #print(b)
    
print(len(basis))
print(len(allSpinsToBasis))
#print(allSpinsToBasis)

35
256


In [5]:
rep = []
for g in oh.elements:
  rep.append(makeRepMat(g, basis, allSpinsToBasis))

In [6]:
fvg.representation_checks.is_valid_rep(rep)

True

In [7]:
metric = np.matrix([[0. for j in range(len(rep[0]))] for i in range(len(rep[0]))], dtype='complex')
for g in rep:
  metric += np.matmul(np.matrix(g).getH(), np.matrix(g))
metric = (metric/len(oh.elements)).round(8)

In [8]:
np.count_nonzero(metric - np.diag(np.diagonal(metric)))

8

In [9]:
metric.shape

(35, 35)

In [10]:
for i,row in enumerate(np.array(metric)):
    for j,val in enumerate(row):
        if not i==j and not np.isclose(0.0,val):
            print("{},{}".format(i,j))


0,20
7,25
9,23
20,0
23,9
25,7
30,34
34,30


In [11]:
metric[0,20]

(0.25+0j)

In [12]:
tot = 0
for irrep in oh.elements[0].irreps:
  ops = operators(irrep, rep, oh)
  tot += len(ops)*len(oh.elements[0].irreps[irrep])
  print("{} ops in {}".format(len(ops), irrep))

print("{} operators across all irreps".format(tot))

1 ops in A1g
0 ops in A2u
3 ops in Eg
2 ops in T1u
3 ops in T2g
0 ops in A1u
0 ops in A2g
2 ops in Eu
1 ops in T1g
2 ops in T2u
35 operators across all irreps


In [13]:
for op in operators('A1g', rep, oh):
    print_vec(op, basis)
    print()
    

(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,0}+(-2+0j)*\bar{u}_{a,1}*\bar{u}_{a,2}*\bar{u}_{a,3}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,1}*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,1}+



In [14]:
for op in operators('A1g', rep, oh):
    print_vec(op, basis)
    print()

(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,0}+(-2+0j)*\bar{u}_{a,1}*\bar{u}_{a,2}*\bar{u}_{a,3}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,1}*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,1}+



In [15]:
for op in operators('Eg', rep, oh,row=0):
    print_vec(op, basis)
    print()
print("-----------------------------")
for op in operators('Eg', rep, oh,row=1):
    print_vec(op.round(2), basis)
    print()

(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,0}*\bar{u}_{a,0}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,1}*\bar{u}_{a,1}*\bar{u}_{a,1}*\bar{u}_{a,1}+(1+0j)*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,2}+(1+0j)*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,3}

(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,0}*\bar{u}_{a,2}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,1}*\bar{u}_{a,1}*\bar{u}_{a,3}*\bar{u}_{a,1}+(1+0j)*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,1}+

(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,2}*\bar{u}_{a,2}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,1}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,1}+

-----------------------------
(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,1}*\bar{u}_{a,1}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,2}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,2}+

(1+0j)*\bar{u}_{a,0}*\bar{u}_{a,1}*\bar{u}_{a,3}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,1}*\bar{u}_{a,1}*\bar{u}_{a,2}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,2}*\bar{u}_{a,3}*\bar{u}_{a,3}*\bar{u}_{a,0}+(1+0j)*\bar{u}_{a,

In [16]:
def print_op_to_file(vec, basis, filename):
  f = open(filename, "w")

  v = copy.deepcopy(basis)
  for i in range(len(vec)):
    v[i].coef *= vec[i]

  text=""
  for i in range(len(v)):
    if not np.isclose(v[i].coef, 0):
        line = ""
        for q in v[i].quarks:
            line += str(q.spin) + " "
        line += "{} {}".format(v[i].coef.real, v[i].coef.imag)
        #print(line)
        line += "\n"
        text += line
  f.write(text[:-1]) # delete the last newline character
  f.close()

In [17]:
import os

In [18]:
irrep="A1g"
for i,op in enumerate(operators(irrep, rep, oh)):
    print_op_to_file(op, basis, os.path.join("OPDATA","baryon_uuuu_{}_{}.txt".format(irrep,i)))

irrep="Eg"
for row in [0,1]:
    for i,op in enumerate(operators(irrep, rep, oh, row)):
        print_op_to_file(op, basis, os.path.join("OPDATA","baryon_uuuu_{}r{}_{}.txt".format(irrep,row,i)))

## Testing rotations of ops

In [19]:
tot = 0
allOps=[]
for irrep in oh.elements[0].irreps:
    ops = operators(irrep, rep, oh)
    for op in ops:
        allOps.append(op)

In [20]:
np.matmul(rep[3],allOps[0])

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+2.77555756e-17j,
        0.00000000e+00+1.38777878e-17j,  1.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  2.22044605e-16-1.38777878e-17j,
       -2.00000000e+00+5.55111512e-17j, -1.11022302e-16+1.38777878e-17j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  1.00000000e+00+0.00000000e+00j,
        0.00000000e+00-1.38777878e-17j,  0.00000000e+00+2.77555756e-17j,
        0.00000000e+00+0.00000000e+00j,  0.00000000

In [21]:
allOps[0]

array([ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
       -2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j])